<a href="https://colab.research.google.com/github/GaryMarmon/vim/blob/master/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import os
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

#MovieLens Data

In [ ]:
import pandas as pd
import numpy as np
import os
#path to data on drive
data_path = '/content/drive/MyDrive/AI_P2/'
movies_filename = '/content/drive/MyDrive/AI_P2/movie.csv'
ratings_filename = '/content/drive/MyDrive/AI_P2/rating.csv'

df_movies = pd.read_csv(
    os.path.join(data_path, movies_filename),
#     movies_filename,
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})
df_ratings = pd.read_csv(
    os.path.join(data_path, ratings_filename),
#     ratings_filename,
    usecols=['userId', 'movieId', 'rating'],
    
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})


In [ ]:
df_ratings=df_ratings[:2000000]
#create sparse MxN matrix
df_movie_features = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0) #NaN values are filled with 0.0

In [ ]:
df_movie_features.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
R = df_movie_features.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [ ]:
from scipy.sparse.linalg import 
# get Nx1 matrix of users cand a 1xN matrix of movies
User_maxtrix, sigma, Movie_matrix = svds(R_demeaned, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
# predict ratings
predicted_ratings = np.dot(np.dot(User_matrix, sigma), Movie_matrix) + user_ratings_mean.reshape(-1, 1)

In [ ]:
preds_df = pd.DataFrame(predicted_ratings, columns = df_movie_features.columns)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
0,-1.032371,0.816785,0.065364,-0.069287,-0.089831,0.488378,-0.417961,0.045865,-0.139051,-0.341773,-0.513422,0.178584,0.069932,-0.034303,-0.045440,0.260362,0.069649,0.285616,0.471102,-0.199864,-0.034729,0.439174,-0.025721,0.213960,0.094146,0.060701,0.057206,-0.044556,1.233433,-0.004471,0.095023,2.282894,-0.011613,0.456059,0.021102,0.520967,0.004016,-0.014233,0.176871,0.043323,...,-0.007710,-0.007637,-0.001437,-0.001974,-0.001901,-0.002341,-0.007710,-0.010073,-0.009284,-0.003201,-0.010073,-0.002748,-0.001605,-0.001605,-0.009607,-0.000441,-0.004316,-0.003167,0.000509,-0.005256,-0.007290,-0.001605,-0.014006,-0.002858,-0.001877,-0.001741,-0.001728,-0.004780,-0.004179,-0.011247,0.000436,-0.000913,-0.005743,-0.006718,-0.004173,-0.002836,-0.009607,-0.006603,-0.001093,-0.002193
1,0.878409,0.018589,0.350089,0.069685,0.219142,0.379299,0.404331,0.008962,0.102664,-0.336462,0.312136,0.088315,0.029474,0.091949,0.045356,0.030741,0.429900,0.061929,-0.042158,0.062091,-0.037037,0.128742,0.110056,0.276818,0.333762,0.034210,0.057615,-0.010574,0.326948,-0.026604,0.111464,1.481751,0.002342,0.073127,-0.019877,0.147415,-0.003927,0.020103,-0.161435,-0.000244,...,-0.003447,0.000291,-0.001468,-0.000321,0.000662,-0.000419,-0.003447,0.001479,0.001308,0.001028,0.001479,0.000548,-0.000801,-0.000801,-0.004802,0.001637,0.000272,0.001330,-0.000974,-0.003719,-0.003486,-0.000801,0.001901,0.000244,-0.000653,-0.000727,0.000495,-0.001496,0.000349,0.001411,0.001779,-0.000374,-0.001408,0.002510,0.002424,-0.000837,-0.004802,0.001166,-0.000476,0.000287
2,2.004502,0.853211,-0.124892,0.033102,-0.197885,0.725845,-0.070362,-0.043636,-0.004803,0.117872,0.675622,-0.011991,0.023256,0.035115,-0.085639,1.054156,-0.394635,-0.095324,-0.392495,-0.006021,0.612254,0.300113,-0.104179,0.892095,0.300978,-0.106822,0.052140,-0.153645,0.799835,-0.006502,-0.099880,2.957939,0.010231,0.690259,-0.046066,-0.073028,0.025883,0.007296,0.748283,-0.027530,...,0.014029,0.002355,-0.005867,0.000009,-0.003435,-0.000388,0.014029,-0.001359,-0.001174,0.000396,-0.001359,0.000377,0.000973,0.000973,0.019519,0.010235,0.000436,0.002968,-0.000737,0.005804,-0.004255,0.000973,0.003951,0.000342,0.000840,0.000906,0.000638,-0.003460,-0.000256,0.003040,0.003959,-0.004786,-0.001086,0.001811,0.005158,0.001003,0.019519,-0.001381,-0.001391,0.000514
3,-0.730042,0.575875,0.316375,-0.043769,0.175545,0.804844,0.024103,0.053929,0.182834,1.027780,0.215090,0.053206,0.031952,0.014743,0.069392,0.533745,-0.540016,0.039601,0.485802,0.259154,0.650262,0.308804,0.244486,0.137633,-0.052208,-0.045443,-0.014521,-0.067064,0.017107,-0.024689,0.171829,0.444833,-0.000831,0.061157,-0.058791,-0.131903,0.003973,0.000392,0.206134,-0.037649,...,0.001747,0.001691,0.002540,0.001675,0.002821,0.001738,0.001747,0.004465,0.004178,0.002712,0.004465,0.001991,0.001990,0.001990,0.001691,0.001270,0.001783,0.002540,0.001151,0.001958,0.004704,0.001990,0.001713,0.001938,0.001969,0.001979,0.001728,0.002784,0.001792,0.001756,0.001220,0.001608,0.002619,0.001078,0.003182,0.001051,0.001691,0.001607,0.002382,0.001786
4,1.487824,1.324306,1.370971,0.112845,1.282825,0.726826,1.453524,0.217864,0.285134,1.278284,1.579988,0.208238,0.088085,0.475251,0.035512,-0.176552,1.851011,0.099256,0.402608,0.031267,0.674256,0.260642,0.019003,0.071229,0.663683,0.228993,0.160756,0.229393,0.067370,-0.015628,0.448436,0.985232,-0.006345,2.396329,0.040165,1.618646,0.005433,0.037594,0.310421,0.106340,...,-0.002487,-0.002947,0.004797,0.000543,0.002156,0.000243,-0.002487,0.003953,0.003598,0.002265,0.003953,0.001517,-0.001509,-0.001509,-0.003786,0.000288,0.002172,-0.001100,0.000004,0.001105,-0.005823

# Recommendations

In [ ]:
def recommend_movies(preds_df, userID, movies_df, ratings_df, num_recommendations=5):
    
    # Get Predictions
    user_row_number = userID - 1 #account for 0 index
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) 
    # Get Titles
    user_data = ratings_df[ratings_df.userId == (userID)]
    watched = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    # Recommend
    recommendations = (movies_df[~movies_df['movieId'].isin(watched['movieId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'movieId',
               right_on = 'movieId').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1]
                      

    return watched, recommendations

In [141]:
watched, recommendations = recommend_movies(preds_df, 2, df_movies, df_ratings, 10)

In [142]:
# movies the user has watched an rated
watched.head(10)

,userId,movieId,rating,title
60,2,3959,5.0,"Time Machine, The (1960)"
15,2,1196,5.0,Star Wars: Episode V - The Empire Strikes Back...
33,2,1974,5.0,Friday the 13th (1980)
45,2,3450,5.0,Grumpy Old Men (1993)
1,2,62,5.0,Mr. Holland's Opus (1995)
46,2,3513,5.0,Rules of Engagement (2000)
26,2,1748,5.0,Dark City (1998)
23,2,1544,5.0,"Lost World: Jurassic Park, The (1997)"
22,2,1356,5.0,Star Trek: First Contact (1996)
21,2,1327,5.0,"Amityville Horror, The (1979)"


In [143]:
# top 10 recommendations based on user ratings
recommendations

,movieId,title
1194,1240,"Terminator, The (1984)"
1159,1200,Aliens (1986)
2448,2571,"Matrix, The (1999)"
345,356,Forrest Gump (1994)
2505,2628,Star Wars: Episode I - The Phantom Menace (1999)
30,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
1157,1198,Raiders of the Lost Ark (Indiana Jones and the...
1061,1097,E.T. the Extra-Terrestrial (1982)
1909,2028,Saving Private Ryan (1998)
1164,1206,"Clockwork Orange, A (1971)"


# Collab filtering and Correlation



In [ ]:
#Combining data
#using previous combined data df
df.head()

,movie_id,customer_id,rating,date,year_of_release,movie_title
0,1,30878,4,2005-12-26,2003.0,Dinosaur Planet
1,1,1245640,3,2005-12-19,2003.0,Dinosaur Planet
2,1,2647871,4,2005-12-30,2003.0,Dinosaur Planet
3,1,883478,4,2005-12-16,2003.0,Dinosaur Planet
4,1,2354601,5,2005-12-10,2003.0,Dinosaur Planet


## Check Data

In [ ]:
#Checking df
df.describe()

,movie_id,customer_id,rating,year_of_release
count,503615.000000,5.036150e+05,503615.000000,503613.000000
mean,2395.323289,1.316493e+06,3.656517,1993.969534
std,1303.515536,7.659066e+05,1.076384,13.339720
min,1.000000,6.000000e+00,1.000000,1916.000000
25%,1256.000000,6.478700e+05,3.000000,1990.000000
50%,2422.000000,1.317124e+06,4.000000,1999.000000
75%,3579.000000,1.980785e+06,4.000000,2003.000000
max,4499.000000,2.649429e+06,5.000000,2005.000000


In [ ]:
#creating average mean rating of movie_title
#ratings = pd.DataFrame(entire_data_df.groupby('movie_title')['rating'].mean())
ratings = pd.DataFrame(df.groupby('movie_title')['rating'].mean())
#adding number of ratings on movie
ratings['number_of_ratings']=df.groupby('movie_title')['rating'].count()
ratings.head()

,rating
movie_title,
'N Sync: 'N the Mix,2.000000
'N Sync: PopOdyssey Live,3.000000
... And God Spoke,3.250000
10,3.345455
10 Minute Solution: Pilates,3.666667


In [ ]:
#making matrix

matrix= df.pivot_table(index='customer_id', columns='movie_title', values='rating')
#matrix= df_movie_features.pivot_table(index='userId', columns='movieId', values='rating')
matrix.head(10)

In [ ]:
#most rated movie
ratings.sort_values('number_of_ratings', ascending=False).head(10)


,rating,number_of_ratings
movie_title,,
Batman Begins,4.117380,11024
The Hitchhiker's Guide to the Galaxy,3.019961,4058
The Bourne Supremacy,3.981936,3543
Bruce Almighty,3.406569,3136
Shrek 2,3.994484,3082
Pirates of the Caribbean: The Curse of the Black Pearl,4.191552,3054
Eternal Sunshine of the Spotless Mind,3.645194,2996
Napoleon Dynamite,3.329880,2992
Sideways,3.299966,2977


##Making Recommendation Similar to "Input"

In [ ]:
#Making Recommendation by input
input='The Bourne Supremacy'
input_rating = matrix[input]



In [ ]:
#Finding similar movie
similar_to_input = matrix.corrwith(input_rating)
similar_to_input.head()

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


movie_title
'N Sync: 'N the Mix           NaN
'N Sync: PopOdyssey Live      NaN
... And God Spoke             NaN
10                            NaN
10 Minute Solution: Pilates   NaN
dtype: float64

##Threshold for min num of ratings

In [ ]:
#create dataframe
sim_input = pd.DataFrame(similar_to_input, columns=['Correlation'])
sim_input.dropna(inplace=True)
sim_input.head()

,Correlation
movie_title,
10 Things I Hate About You,0.242775
101 Dalmatians II: Patch's London Adventure,0.622665
11:14,0.430327
13 Ghosts,0.499046
18 Again,0.128624


In [ ]:
#adding in ratings
sim_input = sim_input.join(ratings['number_of_ratings'])
sim_input.head()

,Correlation,number_of_ratings
movie_title,,
10 Things I Hate About You,0.242775,1347
101 Dalmatians II: Patch's London Adventure,0.622665,150
11:14,0.430327,1053
13 Ghosts,0.499046,230
18 Again,0.128624,28


In [ ]:
sim_input[sim_input['number_of_ratings']>500].sort_values(by='Correlation',ascending=False).head(10)

,Correlation,number_of_ratings
movie_title,,
The Bourne Supremacy,1.000000,3543
The Missing,0.514721,616
The Recruit,0.505970,1098
Firefly,0.497693,648
Taking Lives,0.467793,1242
The Italian Job,0.462978,2167
The Matrix: Revolutions,0.456968,895
Armageddon,0.444099,524
11:14,0.430327,1053
